# Do everything in Python. not Images of EE
[https://github.com/renelikestacos/Google-Earth-Engine-Python-Examples/blob/master/005_EE_Proba-V_NDVI_Time-Series_Analysis.ipynb](https://github.com/renelikestacos/Google-Earth-Engine-Python-Examples/blob/master/005_EE_Proba-V_NDVI_Time-Series_Analysis.ipynb)

    1. Get the Red and NIR bands
    2. Compute NDVI
    3. Form pandas dataframe
    4. Carry on

However, this example is done for one point. We have to deal with space and time.

# Read shapefiles and use them to get images off EE.

In [1]:
import ee
ee.Authenticate()

AttributeError: module 'ee' has no attribute 'Authenticate'

In [2]:
# Initialize the library.
ee.Initialize()

AttributeError: module 'ee' has no attribute 'Initialize'

In [3]:
!pip install geopandas

In [0]:
# Import libraries

from IPython.display import Image
import ee
from pylab import *
import seaborn as sns
from matplotlib.pylab import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose


import numpy as np
import tensorflow as tp
from IPython.display import Image
import folium
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
import datetime
import time
from shapely.geometry import Point, Polygon
import folium

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

print('Folium version: ' + folium.__version__)

Folium version: 0.8.3


# Load Proba-V image collection and point geometry

Selected Location of point is from the Proba-V Footprint X18Y02 in Luxembourg, Europe.

In [0]:
# Set start and end date
startTime = datetime.datetime(2015, 1, 1)
endTime = datetime.datetime(2017, 12, 31)

# Create image collection
collection = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M').filterDate(startTime, endTime)

# Create point in Luxembourg (Proba-V Footprint: X18Y02)
point = {'type':'Point', 'coordinates':[6.134136, 49.612485]};

In [0]:
rr = collection.getRegion(point,500)


# Retrieve information, reshape and calculate NDVI

Retrieving information from point geometry with a buffer of 500m over image collection. Reshaping data and calculating NDVI from RED and NIR band.

In [0]:
info = collection.getRegion(point,500).getInfo()

In [0]:
info[0]

['id', 'longitude', 'latitude', 'time', 'RED', 'NIR', 'BLUE', 'SWIR', 'NDVI', 'SZA', 'SAA', 'SWIRVAA', 'SWIRVZA', 'VNIRVAA', 'VNIRVZA', 'SM', 'TIME']

In [0]:
# Reshape image collection 
header = info[0]
data = array(info[1:])

iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:, iTime].astype(int))]

# List of used image bands
band_list = ['RED', 'NIR']

iBands = [header.index(b) for b in band_list]
yData = data[0:, iBands].astype(np.float)

# Calculate NDVI
red = yData[:,0]
nir = yData[:,1]
ndvi = (nir - red) / (nir + red)

# Reshape NDVI array into Pandas Dataframe

In [0]:
df = pd.DataFrame(data=ndvi, index=list(range(len(ndvi))), columns=['NDVI'])
df = df.interpolate()
df['Date'] = pd.Series(time, index=df.index)
df = df.set_index(df.Date)
df.index = pd.to_datetime(df.index)
df['NDVI']=df['NDVI'].fillna(0)

In [0]:
ndvi.shape

(1089,)

In [0]:
df.shape

(1089, 2)

In [0]:
df.head()

,NDVI,Date
Date,,
2015-01-01,0.000000,2015-01-01
2015-01-02,0.008690,2015-01-02
2015-01-03,0.022652,2015-01-03
2015-01-04,0.017543,2015-01-04
2015-01-05,0.012434,2015-01-05
